# Ingesting a new dataset into the geoDB from a local vector data file

This notebook provides a template for ingesting new data from a local file into a geoDB collection, demonstrating some common techniques and operations which may be needed to prepare data, create a collection, and insert or update database rows. The notebook is not ready to run as is, and must be adapted for the relevant task by adding credentials, the path to the input data, the database name, etc.

## Initial set-up

Some standard imports.

In [ ]:
import geopandas
from xcube_geodb.core.geodb import GeoDBClient
import geopandas
from pyproj.crs import CRS
import os
import numpy as np

Import pyplot and set a sensible figure size.

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = 16,12

Set geoDB credentials.

In [ ]:
os.environ['GEODB_AUTH_CLIENT_ID'] = '[CLIENT ID GOES HERE]'
os.environ['GEODB_AUTH_CLIENT_SECRET'] = '[CLIENT SECRET GOES HERE]'
os.environ['GEODB_AUTH_MODE'] = 'client-credentials'
os.environ['GEODB_AUTH_AUD'] = 'https://geodb.brockmann-consult.de'
os.environ['GEODB_API_SERVER_URL'] = 'https://xcube-geodb.brockmann-consult.de'
os.environ['GEODB_AUTH_DOMAIN'] = 'https://xcube-users.brockmann-consult.de/api/v2'
os.environ['GEOSERVER_SERVER_URL'] = 'https://xcube-geodb.brockmann-consult.de'

## Read and examine the data

Open the data file as a GeoDataFrame. GeoPandas can read (amongst others) shapefiles, zipped shapefile directories, and gpkgs.

In [ ]:
%%time
df = geopandas.read_file("[INPUT FILENAME GOES HERE].zip")

Check that the data looks sane.

In [ ]:
df

Plot the data as a further sanity check.

In [ ]:
%%time
df.plot()

## Make the data suitable for ingestion

First, check the WKT for the projection.

In [ ]:
df.crs.to_wkt()

This could be a non-EPSG CRS. Let's make sure by asking GeoPandas for its EPSG code.

In [ ]:
df.crs.to_epsg() is None

Currently GeoPandas doesn't support non-EPSG CRSs, so if this isn't EPSG we'll have to reproject before ingesting it. The target CRS must be chosen according to the are and application. For demonstration purposes, we use EPSG 31370 below.

In [ ]:
%%time
df_31370 = df.to_crs(epsg=31370)
df_31370

Let's just make sure that the CRS is now as expected.

In [ ]:
df_31370.crs.to_epsg()

Now we need to check the types of the columns in the GeoDataFrame, so we can create corresponding columns in the geoDB collection. We're doing this before `NaN` replacement, because `NaN` replacement can mess with the dtypes – see e.g. <https://stackoverflow.com/questions/59500812/pandas-dataframe-replace-change-dtype-of-columns>. We can use the `dtypes` property to see the types:

In [ ]:
df_31370.dtypes

The `OBJECTID` has type `float64`, which isn't ideal: we know the ID is an integer, and can see it in the quick view of the DataFrame above. Let's cast it to an `int64` before putting it in the database.

In [ ]:
df_31370["OBJECTID"] = df_31370["OBJECTID"].astype("int64")

Now check if there are any `NaN` values in the DataFrame. If so, we'll need to replace them with `None` before ingestion. This check is mainly for curiosity: we can do the replace in any case, and it will be a no-op if there aren't any `NaN`s.

In [ ]:
nan_counts = (df_31370.eq(np.NaN)).sum()
nan_counts

If we do have any `NaN`s, replace them with `None`s. We only do this if there *are* `NaN`s, since it can alter the `dtype`s (see above). This is a fairly crude solution: it would probably be better to do the replace on a per-column basis and only for columns which do contain `NaN`s.

In [ ]:
if nan_counts.sum() > 0:
    df_to_ingest = df_31370.replace({np.NaN: None})
else:
    df_to_ingest = df_31370

## Ingest the data

Now we need to add our data as a geoDB collection. First, connect to the geoDB and check our user ID.

In [ ]:
geodb = GeoDBClient()
geodb.whoami

Make sure that our desired database is available as expected.

In [ ]:
geodb.get_my_databases()

Now let's see what collections we currently have in the database, so we can pick an appropriate name for the new one.

In [ ]:
target_database_name = "my_database"
geodb.get_my_collections(database=target_database_name)

Set a variable for the collection name, used both to create the collection (if required) and insert the data.

In [ ]:
collection_name = "my_collection_name"

Now we need to create a new collection for the data. The database columns are supplied in the dictionary given as the `properties` parameter to `create_collection_if_not_exists`. Note that we don't explicitly specify a `geometry` column: this is included automatically, since it must always be present (otherwise it wouldn't be a geodatabase…). The `int64` and `float64` dtypes map to `int` and `float` in the properties lists, and all the `object` dtypes actually represent strings, so we use the `text` type for them in the database.

In [ ]:
collection = geodb.create_collection_if_not_exists(
    collection=collection_name,
    crs=df_to_ingest.crs.to_epsg(),
    properties={
        "OBJECTID": "int",
        "CAMPAGNE": "text",
        "CULT_COD": "text",
        "CULT_NOM": "text",
        "GROUPE_CUL": "text",
        "SURF_HA":  "float",
    },
    database=target_database_name
)

Finally, we insert the data.

In [ ]:
geodb.insert_into_collection(collection=collection_name, values=df_to_ingest, database=target_database_name, upsert=True)